In [ ]:
!pip install pillow Augmentor requests
!pip install pillow Augmentor
!pip install Pillow
!pip install tensorflow
!pip install Augmentor
!pip install shutil

In [ ]:
!pip install cv2

In [ ]:
import pandas as pd
from PIL import Image
import random
import requests
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.applications.densenet import preprocess_input
import Augmentor
from io import BytesIO
import os
from IPython.display import Image as IPImage, display
import urllib.request
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import shutil
import cv2



In [ ]:
# Assuming the CSV file is named "skin_tone_dataset.csv", adjust the filename as per your dataset.
csv_file = "/content/drive/MyDrive/fitzpatrick17k.csv"

# Load the CSV data into a pandas DataFrame
data = pd.read_csv(csv_file)

# Display the first few rows of the dataset to get an overview of the data
print(data.head())

In [ ]:
# Define the unique Fitzpatrick scale values in the dataset
fitzpatrick_values = [1, 2, 3, 4, 5, 6]

# Create a dictionary to hold data for each client node
client_data_dict = {value: None for value in fitzpatrick_values}

# Split the data and assign to each client based on Fitzpatrick scale values
for value in fitzpatrick_values:
    client_data_dict[value] = data[data['fitzpatrick_scale'] == value]

# Display the size of each client's data
for value, client_data in client_data_dict.items():
    print(f"Client {value} data size: {len(client_data)}")

# If needed, you can save the data for each client into separate CSV files
for value, client_data in client_data_dict.items():
    client_data.to_csv(f"client_{value}_data.csv", index=False)

In [ ]:
##This code for download images from csv files url
import os
import pandas as pd
import requests

# Define the function to download an image
def download_image(url, filename):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Image downloaded from {url} to {filename}")
    else:
        print(f"Error downloading image from {url}. HTTP {response.status_code}")

# Load the CSV data into a pandas DataFrame
csv_file = "/content/drive/MyDrive/client_2_data.csv"
client_data = pd.read_csv(csv_file)

# Folder to save the downloaded images
output_base_folder = "/content/drive/MyDrive/Client2_Images"
if not os.path.exists(output_base_folder):
    os.makedirs(output_base_folder)

# Iterate through the DataFrame and download images category-wise
for index, row in client_data.iterrows():
    url = row['url']
    category = row['three_partition_label']  # Assuming 'three_partition_label' is the category column name

    # Skip if URL is empty
    if pd.isna(url) or not url.strip():
        print(f"Skipping row {index} due to empty URL.")
        continue

    # Skip if category is empty or not one of the expected categories
    if pd.isna(category) or category not in ["non-neoplastic", "malignant", "benign"]:
        print(f"Skipping row {index} due to invalid category: {category}")
        continue

    category_folder = os.path.join(output_base_folder, category)
    if not os.path.exists(category_folder):
        os.makedirs(category_folder)

    filename = os.path.join(category_folder, f"image_{index}.jpg")

    # Validate the URL format
    if not url.startswith(("http://", "https://")):
        print(f"Invalid URL format for index {index}: {url}")
        continue  # Skip this iteration

    # Attempt to download the image
    try:
        download_image(url, filename)
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")


In [ ]:
import os

# Define the path to the Round 1 folder
client_Images = "/content/drive/MyDrive/Client2_Images"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Create a dictionary to store the image counts for each category in Round 1
image_counts = {}

# Iterate through each category and count the images in Round 1
for category in categories:
    category_folder = os.path.join(client_Images, category)
    image_count = len(os.listdir(category_folder))
    image_counts[category] = image_count

# Print the image counts for Round 1
for category, count in image_counts.items():
    print(f"Category: {category}, Image Count: {count}")

In [ ]:
##This code for 50 resize image and splitting for training, validation and testing
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

# Define the base directory where your category-wise images are stored
base_dir = "/content/drive/MyDrive/Client2_Images"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define the path to the output folders
output_folder = "/content/drive/MyDrive/Client2_SplitData"

# Define the target size for resizing
target_size = (256, 256)

# Create the output folders if they don't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through the categories and split the data
for category in categories:
    # Create subfolders for each category
    category_train_folder = os.path.join(output_folder, "train", category)
    category_validation_folder = os.path.join(output_folder, "validation", category)
    category_test_folder = os.path.join(output_folder, "test", category)

    os.makedirs(category_train_folder, exist_ok=True)
    os.makedirs(category_validation_folder, exist_ok=True)
    os.makedirs(category_test_folder, exist_ok=True)

    category_folder = os.path.join(base_dir, category)
    image_file_names = os.listdir(category_folder)

    # Split the data into train, validation, and test sets
    train_files, test_files = train_test_split(image_file_names, test_size=0.2, random_state=42)
    train_files, validation_files = train_test_split(train_files, test_size=0.25, random_state=42)

    # Resize and move files to respective folders
    for filename in train_files:
        src_path = os.path.join(category_folder, filename)
        dst_path = os.path.join(category_train_folder, filename)

        # Open and resize the image
        img = Image.open(src_path)
        img = img.resize(target_size)
        img.save(dst_path)

    for filename in validation_files:
        src_path = os.path.join(category_folder, filename)
        dst_path = os.path.join(category_validation_folder, filename)

        # Open and resize the image
        img = Image.open(src_path)
        img = img.resize(target_size)
        img.save(dst_path)

    for filename in test_files:
        src_path = os.path.join(category_folder, filename)
        dst_path = os.path.join(category_test_folder, filename)

        # Open and resize the image
        img = Image.open(src_path)
        img = img.resize(target_size)
        img.save(dst_path)

print("Image resizing and splitting complete.")


In [ ]:
import os

# Define the base directory where your data is stored
base_dir = "/content/drive/MyDrive/Client6_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Create dictionaries to store counts for each category and dataset
category_counts = {category: {} for category in categories}

# Iterate through the categories and datasets
for category in categories:
    for dataset in ["train", "validation", "test"]:
        category_folder = os.path.join(base_dir, dataset, category)
        num_images = len(os.listdir(category_folder))
        category_counts[category][dataset] = num_images

# Print the counts
for category, counts in category_counts.items():
    print(f"Category: {category}")
    for dataset, num_images in counts.items():
        print(f"  {dataset}: {num_images} images")

# Create a dictionary to store total images for each category
category_total_counts = {category: sum(counts.values()) for category, counts in category_counts.items()}

# Calculate the overall total across all categories
overall_total = sum(category_total_counts.values())

# Print the total images for each category
for category, total_count in category_total_counts.items():
    print(f"Total images in '{category}' category: {total_count}")

# Print the overall total across all categories
print(f"Overall total number of images: {overall_total}")



In [ ]:
!pip install --upgrade tensorflow

In [ ]:
##DenseNet201 Apply Client 1 to 6
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client1_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, "train"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

validation_generator = validation_datagen.flow_from_directory(
    os.path.join(base_dir, "validation"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

# Define the DenseNet201 model with transfer learning
base_model = DenseNet201(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(categories), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 60

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    verbose=1
)
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test accuracy: {test_accuracy}")

# Save the model
model.save("/content/drive/MyDrive/Final_Client1_DenseNet201_60epoch_Model.h5")


In [ ]:
#apply resnet50 Apply Client 1 to 6
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client1_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (224, 224)  # ResNet50's default input size is 224x224

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, "train"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

validation_generator = validation_datagen.flow_from_directory(
    os.path.join(base_dir, "validation"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

# Define the ResNet50 model with transfer learning
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(categories), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 60

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test accuracy: {test_accuracy}")

# Save the model
model.save("/content/drive/MyDrive/Final_Client1_ResNet50_60epoch_Model.h5")


In [ ]:
##Client 1 to 6 Confusion Matrix densenet
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model

# Reload the saved model
saved_model_path = "/content/drive/MyDrive/Final_Client1_DenseNet201_60epoch_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
import tensorflow as tf

# Load the saved model
saved_model_path = "/content/drive/MyDrive/Final_Client1_DenseNet201_60epoch_Model.h5"
loaded_model = tf.keras.models.load_model(saved_model_path)

# Access the last layer's weights
last_layer_weights = loaded_model.layers[-1].get_weights()

# Print the last layer weights
print(last_layer_weights)

In [ ]:
import tensorflow as tf
import numpy as np

# List of paths to the saved models from clients
client_model_paths = [
    "/content/drive/MyDrive/Final_Client1_DenseNet201_60epoch_Model.h5",
    "/content/drive/MyDrive/Final_Client2_DenseNet201_60epoch_Model.h5",
    "/content/drive/MyDrive/Final_Client3_DenseNet201_60epoch_Model.h5",
    "/content/drive/MyDrive/Final_Client4_DenseNet201_60epoch_Model.h5",
    "/content/drive/MyDrive/Final_Client5_DenseNet201_60epoch_Model.h5",
    "/content/drive/MyDrive/Final_Client6_DenseNet201_60epoch_Model.h5"
]

# Load only the last layer of each client's model
client_last_layer_weights = []

for model_path in client_model_paths:
    model = tf.keras.models.load_model(model_path)
    last_layer_weights = model.layers[-1].get_weights()
    client_last_layer_weights.append(last_layer_weights)

# Calculate the average of last layer weights
average_last_layer_weights = np.mean(client_last_layer_weights, axis=0)

# Create a new model with the same architecture as the client models
average_model = tf.keras.models.clone_model(tf.keras.models.load_model(client_model_paths[0]))

# Set the weights of the last layer of the average model to the calculated average
average_model.layers[-1].set_weights(average_last_layer_weights)

# Save the average model
average_model.save("/content/drive/MyDrive/Avg_Weights_Clients_DenseNet201_Model.h5")

In [ ]:
import os

file_path = "/content/drive/MyDrive/Avg_Weights_Clients_DenseNet201_Model.h5"
from keras.models import load_model

loaded_model = load_model(file_path)

# Print model summary
loaded_model.summary()

In [ ]:
# Get the last layer
last_layer = loaded_model.layers[-1]

# Get the weights of the last layer
last_layer_weights = last_layer.get_weights()

# Print the weights
print("Last Layer Weights:")
print(last_layer_weights)

In [ ]:
##Client 1 to 6 Confusion Matrix avg weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client1_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Avg_Weights_Clients_DenseNet201_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Compute avg weights All

import tensorflow as tf
from tensorflow.keras.models import load_model

# Paths to the saved models
model_paths = [
    '/content/drive/MyDrive/Final_Client1_DenseNet201_60epoch_Model.h5',
    '/content/drive/MyDrive/Final_Client2_DenseNet201_60epoch_Model.h5',
    '/content/drive/MyDrive/Final_Client3_DenseNet201_60epoch_Model.h5',
    '/content/drive/MyDrive/Final_Client4_DenseNet201_60epoch_Model.h5',
    '/content/drive/MyDrive/Final_Client5_DenseNet201_60epoch_Model.h5',
    '/content/drive/MyDrive/Final_Client6_DenseNet201_60epoch_Model.h5'
]

# Load models
client_models = [load_model(path) for path in model_paths]

# Get the last layer weights from each model
last_layer_weights = [model.layers[-1].get_weights() for model in client_models]

# Separate weight matrix and bias vector
average_weights_matrix = tf.reduce_mean([weights[0] for weights in last_layer_weights], axis=0)
average_bias_vector = tf.reduce_mean([weights[1] for weights in last_layer_weights], axis=0)

# Create a new model with the same architecture as one of the clients
reference_model = load_model(model_paths[0])

# Set the weights of the last layer to the average weights
reference_model.layers[-1].set_weights([average_weights_matrix, average_bias_vector])

# Save the model with average weights
reference_model.save('/content/drive/MyDrive/Average_Weights_Model.h5')


In [ ]:
##Client 1 Weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client1_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Final_Client1_DenseNet201_60epoch_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 1 average weight
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client1_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Average_Weights_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 2 weight
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client2_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Final_Client2_DenseNet201_60epoch_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 2 average weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client2_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Average_Weights_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 3 weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client3_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Final_Client3_DenseNet201_60epoch_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 3 average weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client3_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Average_Weights_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 4 weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client4_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Final_Client4_DenseNet201_60epoch_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 4 average weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client4_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Average_Weights_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 5 weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client5_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Final_Client5_DenseNet201_60epoch_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 5 average weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client5_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Average_Weights_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 6 weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client6_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Final_Client6_DenseNet201_60epoch_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##Client 6 average weights
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client6_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Average_Weights_Model.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
##20 Round Folder Creation
import os
import shutil
import random
from PIL import Image

# Define the base directory where your downloaded images are stored
base_dir = "/content/drive/MyDrive/Client2_Images"

# Define the number of rounds
num_rounds = 20

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Create the Client1_Folder
client_folder = "/content/drive/MyDrive/Client2_Folder"
os.makedirs(client_folder, exist_ok=True)

# Define the target size for resizing
target_size = (128, 128)

# Iterate through each round
for round_num in range(1, num_rounds + 1):
    round_folder = os.path.join(client_folder, f"Round_{round_num}")
    os.makedirs(round_folder, exist_ok=True)

    # Create a list to track images already added to this round
    added_images = []

    # Iterate through each category
    for category in categories:
        category_folder = os.path.join(base_dir, category)
        round_category_folder = os.path.join(round_folder, category)
        os.makedirs(round_category_folder, exist_ok=True)

        # List all images in the category folder
        category_images = os.listdir(category_folder)

        # Calculate how many images to add to this round (total / num_rounds)
        num_images_to_add = len(category_images) // num_rounds

        # Shuffle the images to ensure randomness
        random.shuffle(category_images)

        # Add unique images to this round
        added_count = 0
        for image in category_images:
            if image not in added_images:
                src_path = os.path.join(category_folder, image)
                dst_path = os.path.join(round_category_folder, image)

                # Open and resize the image
                img = Image.open(src_path)
                img = img.resize(target_size)
                img.save(dst_path)

                # Mark the image as added
                added_images.append(image)
                added_count += 1

            # Check if we've added enough images for this round
            if added_count >= num_images_to_add:
                break

print("Round creation and image resizing complete.")


In [ ]:
import os

# Define the path to the Round 1 folder
round_1_folder = "/content/drive/MyDrive/Client6_Folder/Round_1"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Create a dictionary to store the image counts for each category in Round 1
image_counts = {}

# Iterate through each category and count the images in Round 1
for category in categories:
    category_folder = os.path.join(round_1_folder, category)
    image_count = len(os.listdir(category_folder))
    image_counts[category] = image_count

# Print the image counts for Round 1
for category, count in image_counts.items():
    print(f"Category: {category}, Image Count: {count}")

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras import layers, models

# Set the base directory where client folders are stored
base_clients_dir = "C:\\Users\\Heraa\\Desktop\\fitzlearning"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define the number of rounds
num_rounds = 20

# Define the target size for resizing
target_size = (128, 128)

# Iterate through each round
for round_num in range(1, num_rounds + 1):
    # Initialize a list to store the last layer weights of each client's model for the current round
    last_layer_weights_list = []

    # Iterate through each client folder for the current round
    for client_num in range(1, 7):
        client_folder = os.path.join(base_clients_dir, f"Client{client_num}_Folder")
        current_round_folder = os.path.join(client_folder, f"Round_{round_num}")

        # Build the model for the current round
        base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
        base_model.trainable = False

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(len(categories), activation='softmax')  # Assuming 'categories' is defined
        ])

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define the data directory for training
        train_data_dir = current_round_folder

        # Parameters for training
        batch_size = 32
        epochs = 10

        # Create data generators for training
        train_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical',
            subset='training'
        )

        # Train the model for the current round
        model.fit(train_generator, epochs=epochs)

        # Save the trained model
        model_save_path = os.path.join(client_folder, f"Client{client_num}_Round{round_num}_DenseNet201.h5")
        model.save(model_save_path)
        print(f"Model for Client{client_num} Round_{round_num} trained and saved at: {model_save_path}")

        # Load the saved model weights for the current round
        model.load_weights(model_save_path)

        # Extract and store the last layer weights for computing the average later
        last_layer_weights_list.append(model.layers[-1].get_weights())

    # Compute the average weights of the last layer for the current round across all clients
    average_last_layer_weights = np.mean(last_layer_weights_list, axis=0)

    # Iterate through each client folder for the next round (Round_{round_num+1})
    for client_num in range(1, 7):
        client_folder = os.path.join(base_clients_dir, f"Client{client_num}_Folder")
        next_round_folder = os.path.join(client_folder, f"Round_{round_num+1}")

        # Build the model for the next round and set the average last layer weights as initial weights
        base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
        base_model.trainable = False

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(len(categories), activation='softmax')  # Assuming 'categories' is defined
        ])

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Set the average last layer weights as initial weights
        model.layers[-1].set_weights(average_last_layer_weights)

        # Train the model for the next round
        batch_size = 32
        epochs = 10  # You can adjust the number of epochs as needed
        train_data_dir = next_round_folder
        train_datagen = ImageDataGenerator(rescale=1./255)
        train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical'
        )
        model.fit(train_generator, epochs=epochs)

        # Save the trained model for the next round
        model_save_path = os.path.join(client_folder, f"Client{client_num}_Round{round_num+1}_DenseNet201.h5")
        model.save(model_save_path)
        print(f"Model for Client{client_num} Round_{round_num+1} trained and saved at: {model_save_path}")


In [ ]:
##Client 6
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client4_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Client4_Round20_DenseNet201.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Show the plots
plt.show()



In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras import layers, models
from sklearn.utils.class_weight import compute_class_weight


# Set the base directory where client folders are stored
base_clients_dir = "/content/drive/MyDrive"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define the number of rounds
num_rounds = 20

# Define the target size for resizing
target_size = (128, 128)

# Function to compute class weights
def compute_weights(train_generator):
    class_labels = []
    for _, labels in train_generator:
        class_labels.extend(np.argmax(labels, axis=1))
        if len(class_labels) >= len(train_generator.classes):
            break

    unique_classes = np.unique(class_labels)
    class_weights = compute_class_weight('balanced', classes=unique_classes, y=class_labels)
    class_weight_dict = {class_idx: weight for class_idx, weight in zip(unique_classes, class_weights)}

    return class_weight_dict

# Iterate through each round
for round_num in range(1, num_rounds + 1):
    # Initialize a list to store the last layer weights of each client's model for the current round
    last_layer_weights_list = []

    # Iterate through each client folder for the current round
    for client_num in range(1, 7):
        client_folder = os.path.join(base_clients_dir, f"Client{client_num}_Folder")
        current_round_folder = os.path.join(client_folder, f"Round_{round_num}")

        # Build the model for the current round
        base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
        base_model.trainable = False

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(len(categories), activation='softmax')  # Assuming 'categories' is defined
        ])

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define the data directory for training
        train_data_dir = current_round_folder

        # Parameters for training
        batch_size = 32
        epochs = 1

        # Create data generators for training with data augmentation
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical',
            subset='training'
        )

        # Compute class weights
        class_weights = compute_weights(train_generator)

        # Train the model for the current round with class weights
        model.fit(train_generator, epochs=epochs, class_weight=class_weights)

        # Save the trained model
        model_save_path = os.path.join(client_folder, f"Client{client_num}_Round{round_num}_DenseNet201.h5")
        model.save(model_save_path)
        print(f"Model for Client{client_num} Round_{round_num} trained and saved at: {model_save_path}")

        # Load the saved model weights for the current round
        model.load_weights(model_save_path)

        # Extract and store the last layer weights for computing the average later
        last_layer_weights_list.append(model.layers[-1].get_weights())

    # Compute the average weights of the last layer for the current round across all clients
    average_last_layer_weights = []
    # Iterate over each element in the last_layer_weights_list
    for weights in zip(*last_layer_weights_list):
        # Compute the mean of the weights for each element across all clients
        average_weights = np.mean(weights, axis=0)
        average_last_layer_weights.append(average_weights)

    # Convert the list of average weights to a numpy array
    average_last_layer_weights = np.array(average_last_layer_weights)

    # Iterate through each client folder for the next round (Round_{round_num+1})
    for client_num in range(1, 7):
        client_folder = os.path.join(base_clients_dir, f"Client{client_num}_Folder")
        next_round_folder = os.path.join(client_folder, f"Round_{round_num+1}")

        # Build the model for the next round and set the average last layer weights as initial weights
        base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
        base_model.trainable = False

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(len(categories), activation='softmax')  # Assuming 'categories' is defined
        ])

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Set the average last layer weights as initial weights
        model.layers[-1].set_weights(average_last_layer_weights)

        # Train the model for the next round with class weights
        train_data_dir = next_round_folder
        train_datagen = ImageDataGenerator(rescale=1./255)
        train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical'
        )
        model.fit(train_generator, epochs=epochs, class_weight=class_weights)

        # Save the trained model for the next round
        model_save_path = os.path.join(client_folder, f"Client{client_num}_Round{round_num+1}_DenseNet201.h5")
        model.save(model_save_path)
        print(f"Model for Client{client_num} Round_{round_num+1} trained and saved at: {model_save_path}")


In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


# Reload the saved model
# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client2_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Client2_Round20_DenseNet201.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Parse the classification report to get precision, recall, and f1-score
report_dict = classification_report(true_classes, predicted_classes, target_names=class_labels, output_dict=True)
precision = [report_dict[label]['precision'] for label in class_labels]
recall = [report_dict[label]['recall'] for label in class_labels]
f1_score = [report_dict[label]['f1-score'] for label in class_labels]

# Plot precision, recall, and f1-score
plt.figure(figsize=(10, 5))
x = np.arange(len(class_labels))
bar_width = 0.2
plt.bar(x, precision, bar_width, label='Precision')
plt.bar(x + bar_width, recall, bar_width, label='Recall')
plt.bar(x + 2 * bar_width, f1_score, bar_width, label='F1-score')
plt.xlabel('Classes')
plt.ylabel('Scores')
plt.title('Precision, Recall, and F1-Score by Class')
plt.xticks(x + bar_width, class_labels)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/Client5_SplitData"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for training, validation, and test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

saved_model_path = "/content/drive/MyDrive/Client5_Round20_DenseNet201.h5"
model = load_model(saved_model_path)

# Use the test generator to make predictions
predictions = model.predict(test_generator)

# Convert one-hot encoded predictions back to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the true class labels from the test generator
true_classes = test_generator.classes

# Get class labels for the confusion matrix
class_labels = list(test_generator.class_indices.keys())

# Compute the confusion matrix
confusion_mtx = confusion_matrix(true_classes, predicted_classes)

# Calculate accuracy, precision, recall, and F1-score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Print accuracy, precision, recall, and F1-score
print("\nClassification Report:")
print(report)

# Visualize the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=np.max(cm))
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Display values inside the boxes
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=class_labels)

# Plot precision, recall, and F1-score per class
def plot_metrics_per_class(classification_report):
    metrics_per_class = classification_report.split('\n')[2:-5]
    classes = [m.split()[0] for m in metrics_per_class]
    precision = [float(m.split()[1]) for m in metrics_per_class]
    recall = [float(m.split()[2]) for m in metrics_per_class]
    f1_score = [float(m.split()[3]) for m in metrics_per_class]

    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    ax[0].bar(classes, precision, color='b', alpha=0.5)
    ax[0].set_title('Precision per class')
    ax[0].set_ylabel('Precision')
    ax[0].set_xlabel('Class')
    ax[0].set_xticklabels(classes, rotation=45)
    ax[0].set_ylim([0, 1])

    ax[1].bar(classes, recall, color='g', alpha=0.5)
    ax[1].set_title('Recall per class')
    ax[1].set_ylabel('Recall')
    ax[1].set_xlabel('Class')
    ax[1].set_xticklabels(classes, rotation=45)
    ax[1].set_ylim([0, 1])

    ax[2].bar(classes, f1_score, color='r', alpha=0.5)
    ax[2].set_title('F1-Score per class')
    ax[2].set_ylabel('F1-Score')
    ax[2].set_xlabel('Class')
    ax[2].set_xticklabels(classes, rotation=45)
    ax[2].set_ylim([0, 1])

    plt.tight_layout()
    plt.show()

# Plot precision, recall, and F1-score per class
plot_metrics_per_class(report)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

def evaluate_model(model_path):
    # Load the model
    model = load_model(model_path)

    # Load test data
    test_generator = test_datagen.flow_from_directory(
        os.path.join(base_dir, "Client1_SplitData", "test"),
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical"
    )

    # Use the test generator to make predictions
    predictions = model.predict(test_generator)

    # Convert one-hot encoded predictions back to class labels
    predicted_classes = np.argmax(predictions, axis=1)

    # Get the true class labels from the test generator
    true_classes = test_generator.classes

    # Get class labels for the confusion matrix
    class_labels = list(test_generator.class_indices.keys())

    # Compute the confusion matrix
    confusion_mtx = confusion_matrix(true_classes, predicted_classes)

    # Calculate accuracy, precision, recall, and F1-score
    report = classification_report(true_classes, predicted_classes, target_names=class_labels)

    # Print the confusion matrix
    print("Confusion Matrix:")
    print(confusion_mtx)

    # Print accuracy, precision, recall, and F1-score
    print("\nClassification Report:")
    print(report)

    # Plot the confusion matrix
    plot_confusion_matrix(confusion_mtx, classes=class_labels)

    # Plot precision, recall, and F1-score per class
    plot_metrics_per_class(report)

# Paths to models for different clients
model_paths = [
    "/content/drive/MyDrive/Client1_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client2_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client3_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client4_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client5_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client6_Round20_DenseNet201.h5"
]

# Evaluate each model
for model_path in model_paths:
    print("Evaluation for:", model_path)
    evaluate_model(model_path)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the base directory where your split data is stored
base_dir = "/content/drive/MyDrive/"

# Define the categories
categories = ["non-neoplastic", "malignant", "benign"]

# Define data generators for test sets
batch_size = 32
img_size = (128, 128)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

def evaluate_model(model_path, test_data_dir):
    # Load the model
    model = load_model(model_path)

    # Load test data
    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical"
    )

    # Use the test generator to make predictions
    predictions = model.predict(test_generator)

    # Convert one-hot encoded predictions back to class labels
    predicted_classes = np.argmax(predictions, axis=1)

    # Get the true class labels from the test generator
    true_classes = test_generator.classes

    # Get class labels for the confusion matrix
    class_labels = list(test_generator.class_indices.keys())

    # Compute the confusion matrix
    confusion_mtx = confusion_matrix(true_classes, predicted_classes)

    # Calculate accuracy, precision, recall, and F1-score
    report = classification_report(true_classes, predicted_classes, target_names=class_labels)

    # Print the confusion matrix
    print("Confusion Matrix:")
    print(confusion_mtx)

    # Print accuracy, precision, recall, and F1-score
    print("\nClassification Report:")
    print(report)

    # Plot the confusion matrix
    plot_confusion_matrix(confusion_mtx, classes=class_labels)

    # Plot precision, recall, and F1-score per class
    plot_metrics_per_class(report)

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_metrics_per_class(classification_report):
    metrics_per_class = classification_report.split('\n')[2:-5]
    classes = [m.split()[0] for m in metrics_per_class]
    precision = []
    recall = []
    f1_score = []

    for m in metrics_per_class:
        t = m.split()
        if len(t) > 3:
            precision.append(float(t[1]))
            recall.append(float(t[2]))
            f1_score.append(float(t[3]))

    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    ax[0].bar(classes, precision, color='b', alpha=0.5)
    ax[0].set_title('Precision per class')
    ax[0].set_ylabel('Precision')
    ax[0].set_xlabel('Class')
    ax[0].set_xticklabels(classes, rotation=45)
    ax[0].set_ylim([0, 1])

    ax[1].bar(classes, recall, color='g', alpha=0.5)
    ax[1].set_title('Recall per class')
    ax[1].set_ylabel('Recall')
    ax[1].set_xlabel('Class')
    ax[1].set_xticklabels(classes, rotation=45)
    ax[1].set_ylim([0, 1])

    ax[2].bar(classes, f1_score, color='r', alpha=0.5)
    ax[2].set_title('F1-Score per class')
    ax[2].set_ylabel('F1-Score')
    ax[2].set_xlabel('Class')
    ax[2].set_xticklabels(classes, rotation=45)
    ax[2].set_ylim([0, 1])

    plt.tight_layout()
    plt.show()

# Paths to models for different clients
model_paths = [
    "/content/drive/MyDrive/Client1_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client2_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client3_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client4_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client5_Round20_DenseNet201.h5",
    "/content/drive/MyDrive/Client6_Round20_DenseNet201.h5"
]

# Test data directories for different clients
test_data_dirs = [
    os.path.join(base_dir, "Client1_SplitData", "test"),
    os.path.join(base_dir, "Client2_SplitData", "test"),
    os.path.join(base_dir, "Client3_SplitData", "test"),
    os.path.join(base_dir, "Client4_SplitData", "test"),
    os.path.join(base_dir, "Client5_SplitData", "test"),
    os.path.join(base_dir, "Client6_SplitData", "test")
]

# Evaluate each model for each client
for model_path, test_data_dir in zip(model_paths, test_data_dirs):
    print("Evaluation for:", model_path)
    evaluate_model(model_path, test_data_dir)

